In [18]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки
<br>|-- payment_type = тип оплаты

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [3]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [4]:
spark

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [5]:
df = spark.read.parquet("./taxi_data.parquet")

№1 Посчитайте количество загруженных строк.

In [8]:
df.count()

2540712

In [6]:
df.show()

+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare| tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75| 2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0|  3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0|  0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25|  2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0|  0.0|       5.0|        Cash|
|   4666| 2016-12-30 18:00:00|2016-12-30 18:45:00|        2400|      18.2|46.75|10.15|      61.4| Credit Card|
|

Посмотрим схему данных:

In [9]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.corr">corr</a> & <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cov">cov</a>

In [15]:
df.corr("trip_miles", "trip_total")

0.44816152497587586

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe">describe</a>

In [16]:
df.describe(["trip_total", "trip_miles"]).show()

+-------+------------------+------------------+
|summary|        trip_total|        trip_miles|
+-------+------------------+------------------+
|  count|           2540672|           2540677|
|   mean|15.913560215564042|3.0005873828090266|
| stddev|30.546699217618237|  5.25716922943536|
|    min|               0.0|               0.0|
|    max|           9276.69|             900.0|
+-------+------------------+------------------+



№4 Найдите самый НЕ популярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy">groupBy</a> <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [19]:
df.groupBy("payment_type").count().orderBy("count").show()

+------------+-------+
|payment_type|  count|
+------------+-------+
|    Way2ride|      3|
|       Pcard|    878|
|      Prcard|    968|
|     Dispute|   1842|
|     Unknown|   5180|
|   No Charge|  12843|
| Credit Card|1108843|
|        Cash|1410155|
+------------+-------+



№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [21]:
df.groupBy("taxi_id").count().orderBy("count", ascending = False).show()

+-------+-----+
|taxi_id|count|
+-------+-----+
|    316| 2225|
|   6591| 2083|
|   5071| 2080|
|   8740| 2067|
|   6008| 2033|
|   8629| 2024|
|   1462| 2007|
|    375| 1986|
|   8751| 1938|
|   5357| 1930|
|   8264| 1909|
|   1168| 1809|
|   1946| 1803|
|    336| 1799|
|   1521| 1799|
|   3253| 1764|
|   8561| 1760|
|   8344| 1743|
|   8496| 1742|
|   6482| 1740|
+-------+-----+
only showing top 20 rows



№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where">where</a>

In [30]:
df.filter(df.payment_type == "Cash").groupBy().avg().show()

+-----------------+-----------------+------------------+------------------+--------------------+------------------+
|     avg(taxi_id)|avg(trip_seconds)|   avg(trip_miles)|         avg(fare)|           avg(tips)|   avg(trip_total)|
+-----------------+-----------------+------------------+------------------+--------------------+------------------+
|4360.783044157716|712.4986067049796|2.2804813271493494|11.194248598877067|0.003325194540318936|12.035261470840307|
+-----------------+-----------------+------------------+------------------+--------------------+------------------+



№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [48]:
df2 = df.groupBy("taxi_id").sum("trip_miles").withColumnRenamed("sum(trip_miles)", "totalSum")
df2.filter(df2.totalSum > 1000).count()

2860

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [57]:
df.orderBy("trip_seconds", ascending = False).show()

+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+
|   5667| 2016-11-04 21:30:00|2016-11-05 21:30:00|       86399|       0.0|3.25| 0.0|      4.75|        Cash|
|   4161| 2016-11-14 16:00:00|2016-11-15 16:00:00|       86399|       0.0|3.25| 0.0|      3.25|        Cash|
|   1954| 2016-11-03 00:15:00|2016-11-04 00:15:00|       86399|       0.0|3.25| 0.0|      3.25|        Cash|
|   4219| 2016-11-08 16:00:00|2016-11-09 16:00:00|       86392|       0.0|3.25| 0.0|      3.25|        Cash|
|   4551| 2016-11-03 16:15:00|2016-11-04 16:15:00|       86389|       0.0|3.25| 0.0|      3.25|        Cash|
|   5327| 2016-11-01 14:00:00|2016-11-02 14:00:00|       86389|       0.0|3.25| 0.0|      3.25|        Cash|
|   8129| 2016-11-1

№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.
<br>Отсеките неизвестные машины (не определенный taxi_id).

In [70]:
df.filter(df.taxi_id.isNotNull()).groupBy("taxi_id").sum("trip_total").agg({"sum(trip_total)" : "avg"}).show()

+--------------------+
|avg(sum(trip_total))|
+--------------------+
|   8218.856265256312|
+--------------------+



№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hour">hour</a>

In [71]:
from pyspark.sql.functions import hour

In [82]:
df.withColumn("hourCol", hour(df.trip_start_timestamp)).groupBy("hourCol").count().orderBy("count", ascending = False).show()

+-------+------+
|hourCol| count|
+-------+------+
|     18|181127|
|     19|173779|
|     17|169886|
|     16|156519|
|     20|146602|
|     15|144400|
|     14|141347|
|     13|140899|
|     12|136580|
|     21|126776|
|     11|121196|
|      9|118125|
|     22|116996|
|     10|114401|
|      8| 97980|
|     23| 97774|
|      0| 75971|
|      1| 62165|
|      7| 58212|
|      2| 49114|
+-------+------+
only showing top 20 rows



№11 Сколько поездок началось во второй четверти дня?

In [91]:
df.\
withColumn("hourCol", hour(df.trip_start_timestamp)).\
filter("hourCol >= 6 and hourCol < 12").\
groupBy().count().show()

+------+
| count|
+------+
|538737|
+------+



№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast">cast</a>

In [98]:
from pyspark.sql.types import DateType

In [107]:
df.withColumn("date", df.trip_end_timestamp.cast("Date")).groupBy("date").sum("tips").orderBy("sum(tips)", ascending = False).show()

+----------+------------------+
|      date|         sum(tips)|
+----------+------------------+
|2016-11-03|110102.37000000013|
|2016-11-09|106187.87999999986|
|2016-11-16| 99993.77000000038|
|2016-12-01| 97954.88000000014|
|2016-11-10| 97263.90000000063|
|2016-11-04| 96244.83000000005|
|2016-11-17| 95188.28000000052|
|2016-12-08| 94128.17000000039|
|2016-11-08| 90394.94000000024|
|2016-12-15| 89820.06000000052|
|2016-11-18|  89685.4800000007|
|2016-11-02| 89175.40000000023|
|2016-11-30| 88441.87000000021|
|2016-11-07| 87743.85000000036|
|2016-11-15| 85591.05999999978|
|2016-12-14| 85584.15000000011|
|2016-11-11| 85330.95000000014|
|2016-12-09| 84563.74999999993|
|2016-12-07| 83296.74000000034|
|2016-12-16| 81553.82000000037|
+----------+------------------+
only showing top 20 rows



№13 Сколько было заказов в дату с наибольшим спросом?

In [109]:
df.withColumn("date", df.trip_start_timestamp.cast("Date")).groupBy("date").count().orderBy("count", ascending = False).show()

+----------+-----+
|      date|count|
+----------+-----+
|2016-11-03|61259|
|2016-12-16|59137|
|2016-12-09|58583|
|2016-12-15|57084|
|2016-11-04|56800|
|2016-12-08|56786|
|2016-11-18|56382|
|2016-12-02|55914|
|2016-12-01|55298|
|2016-12-14|53096|
|2016-11-02|51677|
|2016-11-05|51341|
|2016-11-10|50970|
|2016-11-30|50904|
|2016-11-09|50894|
|2016-11-17|50849|
|2016-11-11|50148|
|2016-11-16|49653|
|2016-12-07|49653|
|2016-11-08|48566|
+----------+-----+
only showing top 20 rows



Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [110]:
df_car = spark.read.parquet("taxi_cars_data.parquet")

In [111]:
df_car.show()

+-------+-------------------+
|taxi_id|          car_model|
+-------+-------------------+
|   1159|       Toyota Prius|
|   7273|Ford Crown Victoria|
|   2904|        Honda Civic|
|   3210|        Ford Fusion|
|   2088|       Toyota Camry|
|   4821|Ford Crown Victoria|
|   2069|    Hyundai Elantra|
|   6240|       Toyota Camry|
|    296|     Hyundai Sonata|
|   2136|Ford Crown Victoria|
|   1436|     Toyota Corolla|
|   1090|     Toyota Corolla|
|   7711|        Lincoln MKZ|
|   1572|Ford Crown Victoria|
|   3959|     Chevrolet Aveo|
|   5645|  Chevrolet Lacetti|
|   5149|            Audi A7|
|   6366|     Hyundai Sonata|
|    451|     Hyundai Accent|
|   1669|           Kia Ceed|
+-------+-------------------+
only showing top 20 rows



№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split">split</a>

In [112]:
from pyspark.sql.functions import split

In [127]:
df_car.withColumn("carModel", split(df_car.car_model, " ")[0]).groupBy("carModel").count().orderBy("count", ascending = False).show()

+----------+-----+
|  carModel|count|
+----------+-----+
|      Ford| 1484|
|   Hyundai|  792|
|    Toyota|  691|
| Chevrolet|  473|
|       Kia|  265|
|      Audi|  250|
|   Lincoln|  247|
|     Honda|  246|
|Volkswagen|  244|
|    Nissan|  225|
+----------+-----+



№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join</a>

In [133]:
df.join(df_car, "taxi_id").groupBy("car_model").count().orderBy("count", ascending = False).show()

+-------------------+------+
|          car_model| count|
+-------------------+------+
|Ford Crown Victoria|388682|
|     Hyundai Accent|150764|
|           Kia Ceed|143649|
|     Hyundai Sonata|141570|
|        Ford Mondeo|135466|
|    Hyundai Elantra|134722|
|        Honda Civic|133848|
|            Audi A7|129168|
|  Chevrolet Lacetti|128803|
|     Toyota Corolla|125434|
|        Lincoln MKZ|124616|
|       Toyota Camry|123249|
|    Volkswagen Golf|118449|
|        Ford Taurus|116836|
|        Ford Fusion|113828|
|       Toyota Prius|113099|
|     Chevrolet Aveo|109326|
|      Nissan Altima|108038|
+-------------------+------+



Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [12]:
# Ваш код с coalesce(1)

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [13]:
# Ваш код с printSchema() для DataFrame из csv

Не забудьте посетить SparkUI и изучить историю ваших задач.

In [ ]:
spark